# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)

In [ ]:
!pip install conllutils
!pip install conll_df

  Created wheel for conllutils: filename=conllutils-1.1.4-cp37-none-any.whl size=17696 sha256=c3b96ebaa6b11c57f3954a2d5c39bd5c4ab73c8e574fc580f5881f0390d804bd
  Stored in directory: /root/.cache/pip/wheels/94/08/3a/74fab1fdde78b548f26b2de930af3cee81a592fafe9e758a57
Successfully built conllutils
  ERROR: Failed building wheel for conll-df
  Running setup.py clean for conll-df
Failed to build conll-df
    Running setup.py install for conll-df ... done


In [ ]:
import numpy as np
import operator
import nltk
import pandas as pd
import conllutils
import random
from collections import Counter
from conll_df import conll_df
from nltk.tag import tnt 

**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.



In [ ]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM
%cd /content/UD_English-GUM/
!git checkout 2c8b062269f2d2d3d62405c82d8c25cf24f705dd
%cd ..

Cloning into 'UD_English-GUM'...
remote: Enumerating objects: 2526, done.
remote: Counting objects: 100% (1537/1537), done.
remote: Compressing objects: 100% (667/667), done.
remote: Total 2526 (delta 1347), reused 1055 (delta 870), pack-reused 989
Receiving objects: 100% (2526/2526), 16.21 MiB | 9.40 MiB/s, done.
Resolving deltas: 100% (2237/2237), done.
/content/UD_English-GUM
Note: checking out '2c8b062269f2d2d3d62405c82d8c25cf24f705dd'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2c8b062 Updated statistics.
/content


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

In [ ]:
# Your code goes here
train_path = "UD_English-GUM/en_gum-ud-train.conllu"
#train_path = 'previous/en_gum-ud-train.conllu'
test_path = 'UD_English-GUM/en_gum-ud-test.conllu'
dev_path = 'UD_English-GUM/en_gum-ud-dev.conllu'

#column x is upos tag
file = conll_df(train_path, file_index=False)
train = file[['w', 'x']].copy()

file = conll_df(test_path, file_index=False)
test = file[['w', 'x']].copy()

file = conll_df(dev_path, file_index=False)
dev = file[['w', 'x']].copy()

dev.head()

w     x
s i                     
1 1   Introduction  NOUN
2 1       Research  NOUN
  2             on   ADP
  3  adult-learned   ADJ
  4         second   ADJ

**Part 2**

Write a class **simple_tagger**, with methods *train* and *evaluate*. The method *train* receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method *evaluate* receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.


In [ ]:
class simple_tagger:
  def __init__(self):
    self.tagger = {}
    self.most_freq_tag = ''

  def train(self, data):
    train_df = data.copy()
    train_df['count'] = 1

    new = train_df.groupby(['w','x'])['count'].count().reset_index()
    new.dropna(axis = 'rows', inplace = True)

    df = new.groupby(['w'])['count'].max().reset_index()
    df = df.merge(new, how='left', on=['w','count']) 
    df = df[['w','x']]
    d = df.set_index('w')['x'].to_dict()

    self.tagger = d
    self.most_freq_tag = Counter(d.values()).most_common(1)[0][0]
    return d

  def evaluate(self, data):
    num_sent = data.groupby('s').count().shape[0]
    data['counter'] = 0
    
    def compare_tags(row):
      word = row[0]
      t = row[1]
      d = self.tagger
      if word in d:
        if t == d[word]:
          row['counter'] = 1
      else:
        if t == self.most_freq_tag:
          row['counter'] = 1
      return row['counter'] 

    # Word accuracy
    data['counter'] = data.apply(compare_tags, axis = 1)
    word_count = sum(data['counter'])

    # Sentence accuracy
    df =  data['counter'].groupby(level=[0]).count() - data['counter'].groupby(level=[0]).sum() 
    # If df = 0 then all words in sentence have correct tag
    sent_count = df[df == 0].count()

    w_accuracy = word_count/data.shape[0]
    s_accurary = sent_count/num_sent
    return w_accuracy, s_accurary 

In [ ]:
tag = simple_tagger()
tag.train(train)
wtest, stest = tag.evaluate(test)
wdev, sdev = tag.evaluate(dev)

#For Part 4
save_results_simple_test = ['Simple Tagger', wtest, stest]
save_results_simple_dev = ['Simple Tagger', wdev, sdev]

print(f'Word accuracy on test data: {wtest} \nSentence accuracy on test data: {stest}')

Word accuracy on test data: 0.8460379254049981 
Sentence accuracy on test data: 0.1898876404494382


**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.


In [ ]:
class hmm_tagger:
  def __init__(self):
    self.A = np.array([])
    self.B = np.array([])
    self.Pi = np.array([])
    self.words = {}

  def train(self, data):
    unique_tags = list(data['x'].unique())

    # Matrix A
    num_sent = data.groupby('s').count().shape[0]
    
    tr = data.loc[1]
    df = pd.DataFrame()
    df['a'] = tr['x'][0:-1]
    df['b'] =  tr['x'].shift(-1)
    df['count'] = int(1)

    for j in range(2, num_sent + 1): 
      tr = data.loc[j]
      if tr.shape[0] > 1:
        df3 = pd.DataFrame()
        df3['a'] = tr['x'][0:-1]
        df3['b'] =  tr['x'].shift(-1)
        df3['count'] = int(1)
        df = pd.concat([df, df3], axis=0)

    tag_probability = df.groupby(['a','b']).count()
    tag_probability.sort_index(inplace = True)
    new = pd.DataFrame(tag_probability)
    new.reset_index(inplace = True)
    pivoted = new.pivot(index="a", columns="b")
    pivoted = pivoted.fillna(0)
    s = pivoted.sum(axis = 1)

    for i in range(pivoted.shape[0]):
      pivoted.iloc[i,:] = pivoted.iloc[i,:]/s[i]

    self.A = pivoted

    # Matrix B
    new_df = data
    new_df['count'] = 1
    saved = new_df.groupby(['x','w']).count()
    saved.sort_index(inplace = True)
    saved.columns
    new = pd.DataFrame(saved)

    new.reset_index(inplace = True)
    pivoted = new.pivot(index="x", columns="w")
    pivoted = pivoted.fillna(0)
    s = pivoted.sum(axis = 1)

    for i in range(pivoted.shape[0]):
      pivoted.iloc[i,:] = pivoted.iloc[i,:]/s[i]
    
    self.B = pivoted

    # Pi Matrix
    self.Pi = [1/len(unique_tags) for i in unique_tags]
    self.Pi = np.array(self.Pi)

    # Unique words of train set
    words = []
    for i in pivoted.columns:
      words.append(i[1])

    # Dictionary that maps every unique word in the corpus to a serial numeric id 
    words_dict = {words[x]:x for x in range(len(words))}
    self.words = words_dict

    return self.A, self.B, self.Pi

  def evaluate(self, data):
    unique_works = list(data['w'].unique())
    num_sent = data.groupby('s').count().shape[0]
    best_sequence = []

    test_data = data.copy()
    df1 = pd.DataFrame()
    curr_row = 0

    for i in range(1,num_sent+1):
      df2 = test_data.iloc[curr_row:test_data.loc[(i)].shape[0]+curr_row,:]
      curr_row += test_data.loc[(i)].shape[0]
      sentence = list(test_data.loc[(i)]['w'])
      observation = []
      for word in sentence:
        if word in self.words:
           observation.append(self.words[word])
        else:
            r = random.randint(0,len(self.words))  
            observation.append(r)

      best_sequence = viterbi(np.array(observation), self.A.to_numpy(), self.B.to_numpy(), self.Pi)
      
      result = []
      for i in best_sequence:
        result.append(self.A.columns[i][1])
      
      df2.insert(2, 'tag', result)
      df1 = pd.concat([df1, df2])

    df1['counter'] = 0

    def compare_tags(row):
      if row['x'] == row['tag']:
        return 1
      else:
        return 0

    # Word accuracy
    df1['counter'] = df1.apply(compare_tags, axis = 1)
    word_count = sum(df1['counter'])

    # Sentence accuracy
    df =  df1['counter'].groupby(level=[0]).count() - df1['counter'].groupby(level=[0]).sum() 
    # If df = 0 then all words in sentence have correct tag
    sent_count = df[df == 0].count()

    w_accuracy = word_count/df1.shape[0]
    s_accurary = sent_count/num_sent
    return w_accuracy, s_accurary 

In [ ]:
# Viterbi
np.seterr(divide = 'ignore')

def viterbi (observations, A, B, Pi):
  obs = len(observations)
  a = A.shape[0]
  v = np.zeros((obs, a))
  v[0, :] = np.log(Pi[0] * B[:, observations[0]])
  prev = np.zeros((obs - 1, a))
  best_sequence = []

  for i in range(1, obs):
    for j in range(a):
        prob = v[i - 1] + np.log(A[:, j]) + np.log(B[j, observations[i]])
        prev[i - 1, j] = np.argmax(prob)
        v[i, j] = np.max(prob)

  v_result = np.zeros(obs)
  last_state = np.argmax(v[obs - 1, :])
  v_result[0] = last_state

  backtrack_index = 1
  for i in range(obs - 2, -1, -1):
      v_result[backtrack_index] = prev[i, int(last_state)]
      last_state = prev[i, int(last_state)]
      backtrack_index += 1
  best_sequence = np.flip(v_result, axis=0)
  best_sequence = [int(s) for s in best_sequence]
  return best_sequence

# A simple example to run the Viterbi algorithm:
#( Same as in presentation "NLP 3 - Tagging" on slide 35)
# A = np.array([[0.3, 0.7], [0.2, 0.8]])
# B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
# Pi = np.array([0.4, 0.6])
# print(viterbi([0, 3, 2, 0], A, B, Pi))
# Expected output: 1, 1, 1, 1

In [ ]:
tagger = hmm_tagger()
A, B, Pi = tagger.train(train)
hmmw, hmms = tagger.evaluate(test)
hmmw2, hmms2 = tagger.evaluate(dev)

#For Part 4
saved_results_hmm_test = ['HMM Tagger', hmmw, hmms]
saved_results_hmm_dev = ['HMM Tagger', hmmw2, hmms2]

print(f'Word accuracy on test data: {hmmw} \nSentence accuracy on test data: {hmms}')

Word accuracy on test data: 0.8314705512997614 
Sentence accuracy on test data: 0.15842696629213482


**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over both, the dev and test datasets. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [ ]:
def prepare_data(filepath):
  output = []
  file = conllutils.read_conllu(filepath)
  for s in file:
    output.append(s)
  return output

train = prepare_data(train_path)
test = prepare_data(test_path)
dev = prepare_data(dev_path)

def prepare_data_for_eval(data):
  return [[(token.lemma, token.upos) for token in sentence] for sentence in data]

def sentence_accuracy(data, tagger):
  num_sentence = len(data)
  sent_counter = 0
  for sentence in data:
    data = [[(token.lemma, token.upos) for token in sentence]]
    result = tagger.evaluate(data)
    if result == 1.0:
      sent_counter += 1
  return sent_counter/num_sentence

In [ ]:
#Load data
train_data = prepare_data(train_path)
test_data = prepare_data(test_path)
dev_data = prepare_data(dev_path)

#Initialize tagger
tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(prepare_data_for_eval(train_data))

#Evaluate
w_accuracy = tnt_pos_tagger.evaluate(prepare_data_for_eval(test_data))
s_accuracy = sentence_accuracy(test_data, tnt_pos_tagger)
memm_test = ["MEMM Tagger",w_accuracy, s_accuracy ]

w_accuracy_d = tnt_pos_tagger.evaluate(prepare_data_for_eval(dev_data))
s_accuracy_d = sentence_accuracy(dev_data, tnt_pos_tagger)
memm_dev = ["MEMM Tagger",w_accuracy_d, s_accuracy_d ]

Print both, word level and sentence level accuracy for all the three taggers in a table.

In [ ]:
# Your code goes here
df_test = pd.DataFrame(data = [save_results_simple_test, saved_results_hmm_test, memm_test], columns = ['Tagger Model', 'Word Accuracy', 'Sentence Accuracy'])
df_dev = pd.DataFrame(data= [save_results_simple_dev, saved_results_hmm_dev, memm_dev], columns = ['Tagger Model', 'Word Accuracy', 'Sentence Accuracy'])

print('Evaluation on test data: \n')
print(df_test.to_string(index=False))
print('\n')
print('Evaluation on dev data: \n')
print(df_dev.to_string(index = False))


Evaluation on test data: 

  Tagger Model  Word Accuracy  Sentence Accuracy
 Simple Tagger       0.846038           0.189888
    HMM Tagger       0.831471           0.158427
   MEMM Tagger       0.827452           0.153933


Evaluation on dev data: 

  Tagger Model  Word Accuracy  Sentence Accuracy
 Simple Tagger       0.856777           0.161990
    HMM Tagger       0.850814           0.164541
   MEMM Tagger       0.851840           0.164541
